# Ejercicio/Tarea

## Elizabeth Solis Díaz 



Aprovecha la capacidad de Dask para realizar cómputo en paralelo para ajustar un modelo para predecir la proporción de propina de un viaje. Realiza búsqueda de hiperparámetros en grid con cross validation. Puedes usar funciones de scikit learn. Recuerda usar el decorador delayed para ejecutar en paralelo.

+ ¿Qué tan rápido es buscar en paralelo comparado con una búsqueda secuencial en python?

Haz lo mismo que arriba, pero utilizando la biblioteca Dask-ML http://dask-ml.readthedocs.io/en/latest/

+ ¿Cómo se comparan los tiempos de ejecución de tu búsqueda con la de Dask ML?

In [16]:
from dask import dataframe
trips_df = dataframe.read_csv("/data/trips.csv")
trips_df.head(3)

,car_type,fare_amount,passenger_count,taxi_id,tip_amount,tpep_dropoff_datetime,tpep_pickup_datetime,trip_distance
0,A,22.0,1,1,4.6,2015-01-03 01:37:02,2015-01-03 01:17:32,6.90
1,A,9.0,1,1,0.0,2015-01-05 23:35:02,2015-01-05 23:25:15,1.81
2,A,7.5,1,1,1.0,2015-01-06 15:22:12,2015-01-06 15:11:45,0.96


In [17]:
# Obtengamos estadisticos descriptivos relevantes de la variable propina
trips_df.tip_amount.describe().compute()

count    9198.000000
mean        1.540099
std         2.180261
min        -0.700000
25%         0.000000
50%         1.050000
75%         2.100000
max        40.000000
dtype: float64

# Supuesto
La manera en la cual obtendremos la propina será:


    tip_amount/(tip_amount + fare_amount)

In [18]:
# Truco para convertir a datetime...
trips_df.tpep_pickup_datetime = trips_df.tpep_pickup_datetime.astype('M8[us]')
trips_df.tpep_dropoff_datetime = trips_df.tpep_dropoff_datetime.astype('M8[us]')

In [19]:
# Agrega una columna para la duración del viaje
trips_df["trip_duration"] = trips_df.tpep_dropoff_datetime - trips_df.tpep_pickup_datetime

In [20]:
# Agrega una columna para el porcentaje de propina 
trips_df["total_fee"] = trips_df.tip_amount + trips_df.fare_amount 
trips_df["tip_percentage"] = trips_df.tip_amount/ trips_df.total_fee 
trips_df.head(5)

,car_type,fare_amount,passenger_count,taxi_id,tip_amount,tpep_dropoff_datetime,tpep_pickup_datetime,trip_distance,trip_duration,total_fee,tip_percentage
0,A,22.0,1,1,4.60,2015-01-03 01:37:02,2015-01-03 01:17:32,6.90,00:19:30,26.60,0.172932
1,A,9.0,1,1,0.00,2015-01-05 23:35:02,2015-01-05 23:25:15,1.81,00:09:47,9.00,0.000000
2,A,7.5,1,1,1.00,2015-01-06 15:22:12,2015-01-06 15:11:45,0.96,00:10:27,8.50,0.117647
3,A,8.5,1,1,1.00,2015-01-08 08:31:23,2015-01-08 08:22:12,1.90,00:09:11,9.50,0.105263
4,A,7.5,1,1,1.66,2015-01-08 12:35:54,2015-01-08 12:26:26,1.00,00:09:28,9.16,0.181223


In [32]:
# Matriz X
X = trips_df[["car_type","fare_amount", "passenger_count", "trip_distance", "total_fee"]]
X.head()

,car_type,fare_amount,passenger_count,trip_distance,total_fee
0,A,22.0,1,6.90,26.60
1,A,9.0,1,1.81,9.00
2,A,7.5,1,0.96,8.50
3,A,8.5,1,1.90,9.50
4,A,7.5,1,1.00,9.16


In [33]:
# Variable Respuesta
y = trips_df[["tip_percentage"]]
y.head()

,tip_percentage
0,0.172932
1,0.000000
2,0.117647
3,0.105263
4,0.181223


In [36]:
import dask_searchcv as dcv
parameters = {'kernel': ['linear', 'rbf'], 'C': [1, 10]}


In [37]:
from dask_glm.datasets import make_regression
X, y = make_regression()
est = LinearRegression()
est.fit(X, y)
est.predict(X)
est.score(X, y)

NameError: name 'LinearRegression' is not defined

In [ ]:
from dklearn.cross_validation import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [12]:
import dask_ml.linear_model


## Bonus

Haz lo mismo utilizando Spark ML

+ ¿Cómo se comparan los tiempos de ejecución de Spark vs Dask?

Usa los datos en s3://dask-data/nyc-taxi/2015/yellow_tripdata_2015-01.csv

+ ¿Cambia alguno de los resultados anteriores?